In [ ]:
import sys, os, h5py
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import matplotlib.dates as mdates
import scipy.signal
import seaborn as sns

%config InlineBackend.figure_format = 'retina'

sns.set(style = 'ticks', font_scale=1.25)

In [ ]:
Sitename = 'Ebo'

proj_root_path='.'

freq  = '2D'
first = '06-16' #'05-18'

extreme_rain = [25, 25, 25, 25] # unit: mm
extreme_rain = [30, 30, 30, 30] # unit: mm
extreme_rain = np.asarray(extreme_rain)

In [ ]:
#==================

dump_t365 = np.arange(1,366)

tas = -1.5 + 1*14.0 * np.sin(2 * np.pi * (dump_t365 + 260) / 365) + 273.15

# spinup_data['air_temperature'] = tas + 273.15

# ----

rsds = 230 + 80 * np.sin(2 * np.pi * (dump_t365 + 260) / 365)

# ----

rh = 0.70 + dump_t365 * 0
wdsp = 1.10 + dump_t365 * 0

# ----

n_snow_days = sum(tas <=273.15)
n_rain_days = sum(tas >273.15)

idx_snw_days = tas * 0.0 #[spinup_data['air_temperature']<273.15]

idx_snw_days[tas <=273.15] = 50 /86400 / n_snow_days 

idx_rain_days = tas * 0.0 #[spinup_data['air_temperature']<273.15]

idx_rain_days[tas >273.15] = 60 /86400 / n_rain_days

# ----

tdew = 243.5*(np.log(rh*100/100)+((17.67*(tas - 273.15))/(243.5+tas - 273.15)))/(17.67-np.log(rh*100/100)-((17.67*(tas-273.15))/(243.5+tas - 273.15)))
evap = 10.0 ** (11.40 - 2353.0 / (tdew + 273.15))

rlds = 1.08 * (1 - np.exp(-(0.01*evap)**(tas / 2016))) * 5.670676E-8 * tas**4

# ----

# plt.plot(dump_t365, tas - 273.15)
plt.plot(dump_t365, idx_snw_days)
plt.plot(dump_t365, idx_rain_days)

print(idx_rain_days.sum()*86400, idx_snw_days.sum()*86400)

In [ ]:
df = pd.DataFrame(index = pd.date_range('1991-01-01', '1991-12-31'), columns = ['TAS','RSDS','RH','WSPD','SNOW','RAIN','PREC'])

df['TAS'] = tas.copy()
df['RSDS'] = rsds.copy()
df['RLDS'] = rlds.copy()
df['RH'] = 70
df['WSPD'] = 1.05
df['PRES'] = 64800
df['SNOW'] = idx_snw_days.copy() / 1000
df['RAIN'] = idx_rain_days.copy() / 1000
df['PREC'] = idx_snw_days/ 1000 + idx_rain_days/ 1000

df

In [ ]:
era5_df_forz = df.copy()

fid = open('test.dat','wt')

fid.write('-'*130+'\n')

fid.write('{:^17s}|{:^12s}|{:^12s}|{:^10s}|{:^10s}|{:^11s}|{:^11s}|{:^15s}\n'.format('date/time','windspeed','temperature','humidity','pressure','shortwave','longwave','precipitation'))

fid.write('{:^17s}|{:^12s}|{:^12s}|{:^10s}|{:^10s}|{:^11s}|{:^11s}|{:^15s}\n'.format('yyyy mm dd hh mi','m s{-1}','K',' %','Pa','W m{-2}','W m{-2}','mm d{-1}'))

fid.write('-'*130+'\n')

for i, date0 in enumerate(era5_df_forz.index):
    
    print(i, date0.year)
    
    fid.write('{:4d} {:02d} {:02d} {:02d} {:02d}  {:^12.3f} {:^12.3f} {:^10.3f} {:^10.1f} {:^11.3f} {:^11.3f} {:^15.3f}\n'.format(date0.year, date0.month, date0.day, date0.hour, date0.minute, 
                                                                     era5_df_forz['WSPD'][i], 
                                                                     era5_df_forz['TAS'][i],
                                                                     era5_df_forz['RH'][i],
                                                                     era5_df_forz['PRES'][i],
                                                                     era5_df_forz['RSDS'][i],
                                                                     era5_df_forz['RLDS'][i],
                                                                     era5_df_forz['PREC'][i] * 86400
                                                                     ))

fid.close()